In [ ]:
# Utils packages
import ast
from pathlib import Path, PurePath
from typing import Union

# Data collector packages
from hentai import Utils, Hentai, Option

# Analysis packages
import pandas as pd
from PIL import Image
from tqdm import tqdm

# Compute packages
import numpy as np
import tensorflow as tf

## Data collection

We will use the information and media collected on the nhentai.net website. Recording each entry in a 'metadata.csv' file that aggregates information about the hentai, including links to download pages.

In [ ]:
N_SAMPLE = 0 # Number of sample to refetch from hentai.
DATA_PATH = "data" # Directory to drop collected data.

In [ ]:
def random_hentai(_: int) -> pd.Series:
    return pd.Series(Utils.get_random_hentai().dictionary(Option.all()))

In [ ]:
# Query the nhentai.net API to get N number of random hentai
hentais_df = pd.Series(range(N_SAMPLE))
hentais_df = hentais_df.apply(random_hentai)

# The metadata CSV file that will contain the raw information about the hentai
metadata_path = Path(DATA_PATH) / "raw" / "metadata.csv"

# Create a CSV file with a header only if the file does not exist
if not metadata_path.is_file():
    hentais_df.to_csv(metadata_path, index=False, header="column_names")
else:
    hentais_df.to_csv(metadata_path, index=False, mode="a", header=False)

print("Number of collected hentais: ", len(hentais_df))

## Read dataset file

In [ ]:
hentais_df = pd.read_csv(
    Path(DATA_PATH) / "raw" / "metadata.csv",
    converters={
        column_name: ast.literal_eval
        for column_name in ["tag", "group", "parody", "character", "artist", "category", "image_urls"]
    }
)
hentais_df

## Download images

In [ ]:
def download_hentai(hentai: pd.Series) -> Hentai:
    # A hentai will be stored based on the gallery ID instead of the medai ID
    hentai_path = Path(DATA_PATH) / "raw" / str(hentai.id)

    # Since downloading a hentai is an expensive operation, we only
    # download when the hentai ID directory does not exist
    if not hentai_path.is_dir():
        hentai = Hentai(hentai.id)
        hentai.download(hentai_path, progressbar=True)
        return True

    return False

downloaded_hentais_df = hentais_df.apply(download_hentai, axis=1)

In [ ]:
print("Number of downloaded hentais :", len(hentais_df) - downloaded_hentais_df.sum())

## Remove duplicates

In [ ]:
hentais_df = hentais_df.drop_duplicates("id")
hentais_df.to_csv(metadata_path, index=False, header="column_names")

## Remove corrupted images

In [ ]:
def hentai_filenames(hentai: pd.Series):
    return [
        str(PurePath(str(hentai["id"])) / Path(url).name)
        for url in hentai["image_urls"]
    ]

hentais_df = hentais_df.assign(filenames=hentais_df.apply(hentai_filenames, axis=1))
hentais_df

In [ ]:
def preprocess_image(filename: str) -> bool:
    image_path = Path(filename)
    image_preproc_dir = Path(DATA_PATH) / "preprocessed" / image_path.parent
    image_preproc_path = image_preproc_dir / Path(image_path.stem + ".jpg")

    # Preprocess only those images that do not exist in
    # the destination dataset for performance reason
    if not image_preproc_path.is_file():
        try:
            image = Image.open(Path(DATA_PATH) / "raw" / image_path)
            # Normalize images to be JPEG compatible
            image = image.convert("RGB")
            image_preproc_dir.mkdir(parents=True, exist_ok=True)
            image.save(image_preproc_path, "JPEG")
        except Exception as e:
            print(e)
            return False

    return True

def preprocess_images(hentai: pd.Series) -> bool:
    return [
        preprocess_image(filename)
        for filename in hentai["filenames"]
    ]

tqdm.pandas()
preprocessed_hentais_df = hentais_df.progress_apply(preprocess_images, axis=1)

We will filter every hentais that contain at least one corrupted images, since pages have order dependencies.

In [ ]:
hentais_df = hentais_df[preprocessed_hentais_df.apply(all)]

# The metadata CSV file that will contain the raw information about the hentai
metadata_path = Path(DATA_PATH) / "preprocessed" / "metadata.csv"
hentais_df.to_csv(metadata_path, index=False, header="column_names")

print("Number of preprocessed images :", len(hentais_df))